In [2]:
#This is a modification of the original script for Google Colab
#This script is used to download and run a Minecraft server on a GitHub Codespace
#By @Lucast1574 @lucast_1574
# AVAILABLE TESTED VERSIONS:
# - 1.20.4
# - 1.20.3
# - 1.20.2
# - 1.20.1
# - 1.20
# - 1.19.4
# - 1.19.3
# - 1.19.2
# - 1.19.1
# - 1.19
# - 1.18.2
# - 1.18.1
# - 1.18
# - 1.17.1
# - 1.17
# - 1.16.5
# - 1.16.4
# - 1.16.3
# - 1.15.2
# - 1.14.4
# - 1.13.2
# - 1.12.2
# - 1.11.2
# - 1.10.2
# - 1.9.4
# - 1.8.8


version = '1.12.2' # CHANGE THIS TO THE VERSION YOU WANT TO USE

#Chose server type. Currently available versions:
# - paper (The most optimized server for Spigot/Bukkit)
# - forge (For modded servers)
# - fabric (For modded servers)(Alternative to forge)
# - vanilla (For vanilla servers)
# - snapshot (For testing new features)
# - mohist (For modded servers and plugins)

server_type = 'mohist'

#Here we import the necessary libraries

import os
import requests
import json


server_directory = "Minecraft-server"

# Create the server directory on the codespace

os.makedirs(server_directory, exist_ok=True)
os.chdir(server_directory)

# Internal init...

if server_type == 'paper':
    a = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version)

    b = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]))

    print("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"])
    serverURL = "https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"]

if server_type == 'forge':
    serverURL = "https://serverjars.com/api/fetchJar/modded/forge/" + version

if server_type == 'vanilla':
    serverURL = "https://serverjars.com/api/fetchJar/vanilla/vanilla/" + version

if server_type == 'snapshot':
    serverURL = "https://serverjars.com/api/fetchJar/vanilla/snapshot/" + version
if server_type == 'fabric':
    serverURL = 'https://maven.fabricmc.net/net/fabricmc/fabric-installer/0.11.2/fabric-installer-0.11.2.jar'

if server_type == 'mohist':
    serverURL = "https://serverjars.com/api/fetchJar/modded/mohist/" + version

jar_name = {'paper': 'server.jar', 'fabric': 'fabric-installer.jar', 'forge': 'forge.jar', 'vanilla': 'vanilla.jar', 'snapshot': "snapshot.jar", 'mohist': "mohist.jar"}

print('Downloading the selected server: ')

if serverURL:
    print('Dowloading to github codespaces...')
    r = requests.get(serverURL)

    if r.status_code == 200:
        with open(jar_name[server_type], 'wb') as f:
            f.write(r.content)
    else:
        print('Error '+ str(r.status_code) + '! The version you choose does not work.')


# Run on a specific path depending on the server type

if server_type == 'fabric':
    !java -jar fabric-installer.jar server -mcversion $version -downloadMinecraft

if server_type == 'forge':
    %cd "/workspaces/codespaces-jupyter/Minecraft-server"
    !java -jar forge.jar --installServer

# Saves the configuration file on the server
colabconfig = {"server_type": server_type, "server_version": version}
with open("colabconfig.json", 'w') as config_file:
    json.dump(colabconfig, config_file)

# print a completion message
print('Done!')
# change eula to true
!echo "eula=true" >> eula.txt


Dowloading to github codespaces...


Done!


In [3]:
#import the libraries needed for the server to run

import requests

import time

import os

import re

import json

import glob

import threading



#Update apt cache

!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"

#Asign the files to the directory

os.makedirs("/workspaces/codespaces-jupyter/Minecraft-server", exist_ok=True)

os.chdir("/workspaces/codespaces-jupyter/Minecraft-server")

!ls #List the files in the directory to verify them



#Import the configuration file

config_path = "/workspaces/codespaces-jupyter/Minecraft-server/colabconfig.json"

if os.path.isfile(config_path):

    with open(config_path) as config_file:

        colabconfig = json.load(config_file)

else:

    colabconfig = {"server_type": "generic"} # Use the default configuration if there the configuration file does not exist

    with open(config_path, 'w') as new_config_file:

        json.dump(colabconfig, new_config_file)



# Instal OpenJDK


version = colabconfig["server_version"]

if colabconfig["server_version"] < "1.17":

    !sudo apt-get purge openjdk* > /dev/null 2>&1

    !sudo apt-get install openjdk-8-jre-headless &>/dev/null and print("OpenJDK 8 installed") or print("Failed to install OpenJDK 8.")

else:

    !sudo apt-get purge openjdk* > /dev/null 2>&1

    !sudo apt-get install openjdk-17-jre-headless &>/dev/null and print("OpenJDK 17 installed") or print("Failed to install OpenJDK 17.")


# Check the version of java

java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'

if java_ver[0] == "17":

    print("You are using JAVA 17.")

else:

    print("You are using a downgraded java version. Minecraft 1.17+ might not work.")



#List of server types


jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar', 'forge': 'forge.jar', 'vanilla': 'vanilla.jar', 'snapshot': 'snapshot.jar', 'mohist': "mohist.jar"}

jar_name = jar_list[colabconfig["server_type"]]



# Java arguments.

if colabconfig["server_type"] == "paper":

    server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"

else:

    server_flags = "" # No flags for other server types

#JVM Arguments for the server

memory_allocation = "-Xms10G -Xmx10G"



# tunnel_service for the server

tunnel_service = "ngrok"

print("Using: ", tunnel_service)


if tunnel_service == "ngrok":

    !pip -q install pyngrok #Here you instal pyngrok to use ngrok on the codespace

    from pyngrok import conf, ngrok



    # get your ngrok token from https://dashboard.ngrok.com/auth

    print("Get your authtoken from: https://dashboard.ngrok.com/auth") #Here you get your authtoken from ngrok, it will be a dinamic token, if you want a static token you need to pay for ngrok

    import getpass



    # v - - - - - - - TOKEN - - - - - - - v



    authtoken = "2colTMxco9j6JoyY5nBUpvwCJF4_2LDGysD1eKNKJ3fghbE3P" # <---- Place yout token here

    ! ngrok authtoken $authtoken # log in ngrok



    #Choose your region



    #v - - - - - - - Available Regions - - - - - - - v

    # ap - Asia/Pacific (Singapore)

    # au - Australia (Sydney)

    # eu - Europa (Frankfurt - Alemania)

    # in - India (Mumbai)

    # jp - Japon (Tokyo)

    # sa - South america (São Paulo - Brasil)

    # us - United estates (Ohio)

    conf.get_default().region = 'us' # <--- Place your region here


    # Conect to ngrok

    url = ngrok.connect(25565, 'tcp')

    print('Your server IP: ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))

    #If you got the premiun version of ngrok use the comented code below to get a subdomain

    #subdomain = "my-subdomain" # <--- Place your subdomain here inside the quotes

    #url = ngrok.connect(25565, 'tcp', subdomain=subdomain)

    #print('Your server ip is:  ' + url.replace('tcp://', ''))



print('Launching server...')

def get_users():
    url = 'https://jsonplaceholder.typicode.com/users'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            print("Lista de usuarios:")
            for user in data:
                print(f"Name: {user['name']}, Email: {user['email']}")
        else:
            print(f"Error making the request. Status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"Conexion Error: {e}")

# Funcion for executing get_users() every 10 minutes

def repeat_get_users():
    while True:
        get_users()
        time.sleep(600)  # Wait 10 minutes

# Start the thread for repeat_get_users()

user_thread = threading.Thread(target=repeat_get_users)
user_thread.start()

if colabconfig["server_type"] == "forge":

    version = colabconfig["server_version"]

    if colabconfig["server_version"] < "1.17":

        oldpathtoforge = glob.glob(f"/workspaces/codespaces-jupyter/Minecraft-server/forge-{version}-*.jar")

        if oldpathtoforge: # Checks if the list is not empty

            path = oldpathtoforge[0] # Get the first path from the list

            print(path)

            !java $memory_allocation -jar "{path}" nogui

        else:

            print("No forge universal jar found.")

    else:

        pathtoforge = glob.glob(f"/workspaces/codespaces-jupyter/Minecraft-server/libraries/net/minecraftforge/forge/{version}-/unix_.txt")

        if pathtoforge: # Checks if the list is not empty

            path = pathtoforge[0] # Get the first path from the list

            print(path)

            !java @user_jvm_args.txt "@{path}" "$@"

        else:

            print("No unix_args.txt found.")
else:

    !java $memory_allocation $server_flags -jar $jar_name nogui



# Cicle to keep the server running

while True:
    time.sleep(60)

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
apt cache successfully updated
banned-ips.json		       help.yml			    run.sh
banned-players.json	       libraries		    server-icon.png
bukkit.yml		       logs			    server.properties
colabconfig.json	       minecraft_server.1.12.2.jar  spigot.yml
commands.yml		       mods			    user_jvm_args.txt
config			       mohist-config		    usercache.json
defaultconfigs		       mohist.jar		    usernamecache.json
eula.txt		       ops.json			    whitelist.json
forge-1.12.2-14.23.5.2860.jar  permissions.yml		    world
forge.jar		       plugins
forge.jar.log		       run.bat
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `sudo apt-get install openjdk-8-jre-headless &>/dev/null and print("OpenJDK 8 installed") or print("Failed to install OpenJDK 8.")'
You are using a downgraded java version. Minecraft 1.17+ might not work.
Using:  ngrok
Get your autht